In [ ]:
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np

In [ ]:
ev_base_price = 35000  # Example price in euros
cv_base_price = 30000

ev_maintenance_cost_per_km = 0.05  # €/km
cv_maintenance_cost_per_km = 0.08

ev_depreciation_rate = 0.15  # 15% per year
cv_depreciation_rate = 0.20


In [ ]:
def generate_electricity_prices(mean_price, std_dev, size):
    return np.random.normal(mean_price, std_dev, size)

electricity_prices = generate_electricity_prices(mean_price=0.20, std_dev=0.05, size=1000)


In [ ]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Vehicle Cost Comparison Tool"),

    html.Div([
        html.Label("Vehicle Type"),
        dcc.RadioItems(
            id='vehicle-type',
            options=[
                {'label': 'Electric Vehicle', 'value': 'EV'},
                {'label': 'Combustion Vehicle', 'value': 'CV'}
            ],
            value='EV',
            labelStyle={'display': 'inline-block', 'margin-right': '20px'}
        ),
    ]),

    html.Div([
        html.Label("Annual Mileage (km)"),
        dcc.Input(
            id='annual-mileage',
            type='number',
            value=15000
        ),
    ]),

    html.Div([
        html.Label("Ownership Period (years)"),
        dcc.Input(
            id='ownership-period',
            type='number',
            value=5
        ),
    ]),

    html.Div([
        html.Label("Electricity Price Mean (€/kWh)"),
        dcc.Input(
            id='electricity-price-mean',
            type='number',
            value=0.20
        ),
    ]),

    html.Div([
        html.Label("Electricity Price Std Dev"),
        dcc.Input(
            id='electricity-price-std',
            type='number',
            value=0.05
        ),
    ]),

    html.Button('Calculate', id='calculate-button', n_clicks=0),

    dcc.Graph(id='cost-comparison-graph')
])


In [ ]:
def calculate_ev_costs(annual_mileage, years, mean_price, std_dev):
    efficiency = 15  # kWh per 100 km
    electricity_prices = generate_electricity_prices(mean_price, std_dev, size=len(years))
    total_costs = []
    residual_values = []
    accumulated_cost = ev_base_price
    current_value = ev_base_price

    for i, year in enumerate(years):
        yearly_kwh = (annual_mileage / 100) * efficiency
        energy_cost = yearly_kwh * electricity_prices[i]
        maintenance_cost = annual_mileage * ev_maintenance_cost_per_km
        accumulated_cost += energy_cost + maintenance_cost
        current_value *= (1 - ev_depreciation_rate)
        total_costs.append(accumulated_cost)
        residual_values.append(current_value)

    return total_costs, residual_values

def calculate_cv_costs(annual_mileage, years):
    efficiency = 6  # Liters per 100 km
    fuel_price = 1.50  # €/Liter
    total_costs = []
    residual_values = []
    accumulated_cost = cv_base_price
    current_value = cv_base_price

    for year in years:
        yearly_liters = (annual_mileage / 100) * efficiency
        fuel_cost = yearly_liters * fuel_price
        maintenance_cost = annual_mileage * cv_maintenance_cost_per_km
        accumulated_cost += fuel_cost + maintenance_cost
        current_value *= (1 - cv_depreciation_rate)
        total_costs.append(accumulated_cost)
        residual_values.append(current_value)

    return total_costs, residual_values


In [ ]:
@app.callback(
    Output('cost-comparison-graph', 'figure'),
    Input('calculate-button', 'n_clicks'),
    Input('vehicle-type', 'value'),
    Input('annual-mileage', 'value'),
    Input('ownership-period', 'value'),
    Input('electricity-price-mean', 'value'),
    Input('electricity-price-std', 'value')
)
def update_graph(n_clicks, vehicle_type, annual_mileage, ownership_period, electricity_price_mean, electricity_price_std):
    if n_clicks > 0:
        years = np.arange(1, ownership_period + 1)
        if vehicle_type == 'EV':
            total_costs, residual_values = calculate_ev_costs(
                annual_mileage, years, electricity_price_mean, electricity_price_std)
        else:
            total_costs, residual_values = calculate_cv_costs(annual_mileage, years)

        # Create the figure
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=years, y=total_costs, mode='lines+markers', name='Total Cost'))
        fig.add_trace(go.Scatter(x=years, y=residual_values, mode='lines+markers', name='Residual Value'))
        fig.update_layout(title='Cost and Residual Value Over Time', xaxis_title='Years', yaxis_title='Amount (€)')
        return fig
    else:
        return go.Figure()


In [ ]:
if __name__ == '__main__':
    app.run_server(debug=True)
